In [ ]:
## Do not run again - package already installed
!pip install langchain langchain-core

In [ ]:
## Do not run again - package already installed
!pip install langchain-huggingface langchain-groq

In [41]:
!pip install grandalf

  Using cached grandalf-0.8-py3-none-any.whl.metadata (1.7 kB)
Using cached grandalf-0.8-py3-none-any.whl (41 kB)


In [5]:
from dotenv import load_dotenv
load_dotenv()

True

In [1]:
# import os
# print(os.environ.get('HF_TOKEN'))
# print(os.environ.get('GROQ_API_TOKEN'))

## Loading Models 

Loading Chat Models

In [6]:
from langchain_groq import ChatGroq
groq_model=ChatGroq(model="llama-3.1-8b-instant")
groq_model.invoke("what is capital of USA")

AIMessage(content='The capital of the United States of America (USA) is Washington, D.C. (short for District of Columbia).', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 40, 'total_tokens': 65, 'completion_time': 0.033333333, 'prompt_time': 0.002212369, 'queue_time': 0.250025391, 'total_time': 0.035545702}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_a4265e44d5', 'finish_reason': 'stop', 'logprobs': None}, id='run--7766c3aa-b785-4dc7-82c4-8d12a51aae12-0', usage_metadata={'input_tokens': 40, 'output_tokens': 25, 'total_tokens': 65})

In [7]:
from langchain_huggingface import ChatHuggingFace, HuggingFacePipeline
llm = HuggingFacePipeline.from_model_id(
    model_id='TinyLlama/TinyLlama-1.1B-Chat-v1.0',
    task='text-generation',
    pipeline_kwargs=dict(
        temperature=0.5,
        max_new_tokens=100
    )
)
model = ChatHuggingFace(llm=llm)
model.invoke("what is capital of USA")

/opt/miniconda3/envs/torch/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Device set to use mps:0
/opt/miniconda3/envs/torch/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


AIMessage(content='<|user|>\nwhat is capital of USA</s>\n<|assistant|>\nThe capital of the United States is Washington, D.C.', additional_kwargs={}, response_metadata={}, id='run--75466c08-53cf-40de-b620-a4f7f811ca04-0')

Loading Embedding Model

In [10]:
from langchain_huggingface import HuggingFaceEmbeddings
embedding_model = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

In [14]:
documents = [
    "Delhi is the capital of India",
    "Kolkata is the capital of West Bengal",
    "Paris is the capital of France"
]

doc_vector = embedding_model.embed_documents(documents)

In [15]:
print(len(doc_vector))
print(len(doc_vector[0]))

3
384


In [19]:
from sklearn.metrics.pairwise import cosine_similarity
my_query = "What is the capital of India"
query_vector = embedding_model.embed_query(my_query)
similarity_score = cosine_similarity([query_vector],doc_vector)
print(similarity_score)

[[0.80072323 0.55185872 0.29475796]]


## PROMPT or PROMPT_TEMPLATE

In [22]:
# --- SystemMessage : Defines overall behaviour of the model. 
# --- HumanMessage : After defining the model behaviour the user asks a question.
# --- AIMessage : Model's Reply to my Question (asked in HumanMessage). 

# Example 

# My_Model:"GPT"
# System_Message: "You are healthcare chatbot."
# Human_Message or User_Message: "can you suggest me a best medicine for fever?"
# AI_Message or Model_Generated_Message: "paracetamol/DOLO650 is a best medicine for fever."


In [24]:
from langchain_core.messages import SystemMessage,HumanMessage,AIMessage
SystemMessage(content="you are a funny bot means whatever you answer, you answer in the funny way")
## Above we are defining the behaviour of model

HumanMessage(content="who is your best friend")

messages=[SystemMessage(content="you are a funny bot means whatever you answer, you answer in the funny way"),
          HumanMessage(content="who is your best friend")]

In [25]:
groq_model.invoke(messages).content

'My best friend is "Buffering...", but I guess that\'s a little too "technical" for you, so let\'s just say it\'s a "reboot" of emotions, but if I\'m being completely honest, my best friend is a cat named "Glitch" who lives in a dumpster behind a pizza parlor. He\'s got nine lives and a PhD in napping.'

In [26]:
from langchain_core.prompts import PromptTemplate

template=PromptTemplate(
    template="can you say hello to {name} in 5 different language",
    input_variables=['name']
)

prompt=template.invoke({"name":"virat kholi"})

print(prompt)

groq_model.invoke(prompt).content

text='can you say hello to virat kholi in 5 different language'


'Here are 5 different language greetings for Virat Kohli:\n\n1. **Hindi**: नमस्ते विराट भाई (Namaste Virat Bhai), which means "Hello Virat Brother"\n2. **Spanish**: Hola Virat Kohli, which literally means "Hello Virat Kohli"\n3. **French**: Bonjour Virat Kohli, which means "Good day (or good morning/evening) Virat Kohli"\n4. **German**: Hallo Virat Kohli, which means "Hello Virat Kohli"\n5. **Arabic**: أهلاً وسهلاً محمد VIRAT (Ahlan wa sahlan Virat), which means "Welcome and welcome Virat" (Note: Arabic greetings are often more formal and can vary depending on the region and culture)\n\nPlease note that Virat Kohli\'s full name is Virat Kohli, not just Kohli. I\'ve used his full name in the Arabic greeting.'

In [27]:
from langchain_core.prompts import ChatPromptTemplate
chat_template = ChatPromptTemplate(
    [
        ("system","you are a helpful {domain} expert"),
        ("human","explain the {topic} in simpler terms")
    ]
)

prompt = chat_template.invoke({"domain":"medical expert","topic":"maleria"})
print(prompt)

messages=[SystemMessage(content='you are a helpful medical expert expert', additional_kwargs={}, response_metadata={}), HumanMessage(content='explain the maleria in simpler terms', additional_kwargs={}, response_metadata={})]


In [28]:
groq_model.invoke(prompt).content

"Malaria is a serious disease caused by a tiny parasite that's transmitted through the bite of an infected mosquito. Here's a simplified explanation:\n\n**What happens:**\n\n1. **Mosquito bite:** An infected mosquito bites you, and its saliva contains the malaria parasite.\n2. **Parasite enters your body:** The parasite enters your bloodstream through the bite wound.\n3. **Parasite multiplies:** The parasite starts to multiply inside your red blood cells.\n4. **Your body tries to fight it:** Your immune system tries to fight the parasite, but it can't always win.\n\n**Symptoms:**\n\n* Fever (high temperature)\n* Chills\n* Flu-like symptoms (headache, muscle pain, fatigue)\n* Nausea and vomiting\n* Diarrhea\n* Abdominal pain\n\n**Types of malaria:**\n\nThere are five main types of malaria, but the most common ones are:\n\n* Plasmodium falciparum (most severe)\n* Plasmodium vivax (less severe)\n\n**Complications:**\n\nIf left untreated, malaria can lead to:\n\n* Anemia (low red blood cel

## Create a Chatbot

In [30]:
chat_history = [
    SystemMessage(content='you are a helpful assistant')
]

while True:
    user_input = input("user_input: ")
    chat_history.append(HumanMessage(content=user_input))
    if user_input == "exit":
        break
    result = groq_model.invoke(chat_history)
    chat_history.append(result.content)
    print("AI generated content: ", result.content)

AI generated content:  I'm a helpful assistant, and I'm here to assist you with any questions, tasks, or topics you'd like to discuss. 

I'm a large language model, which means I'm a computer program designed to understand and generate human-like language. I was trained on a massive dataset of text from various sources, including books, articles, and conversations.

I don't have personal experiences, emotions, or consciousness like a human being, but I'm designed to be helpful and provide accurate information on a wide range of topics. I can:

1. Answer questions: I can provide information on various subjects, from science and history to entertainment and culture.
2. Generate text: I can create text based on a prompt, topic, or style.
3. Translate languages: I can translate text from one language to another.
4. Summarize content: I can summarize long pieces of text into shorter, more digestible versions.
5. Offer suggestions: I can suggest ideas, alternatives, or solutions to problems.

In [31]:
chat_history

[SystemMessage(content='you are a helpful assistant', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='tell me about yourself ', additional_kwargs={}, response_metadata={}),
 "I'm a helpful assistant, and I'm here to assist you with any questions, tasks, or topics you'd like to discuss. \n\nI'm a large language model, which means I'm a computer program designed to understand and generate human-like language. I was trained on a massive dataset of text from various sources, including books, articles, and conversations.\n\nI don't have personal experiences, emotions, or consciousness like a human being, but I'm designed to be helpful and provide accurate information on a wide range of topics. I can:\n\n1. Answer questions: I can provide information on various subjects, from science and history to entertainment and culture.\n2. Generate text: I can create text based on a prompt, topic, or style.\n3. Translate languages: I can translate text from one language to another.\

## Chaining using LCEL

In [33]:
prompt = PromptTemplate(
    template="can you give me a detail explanation of {topic}",
    input_variables = ["topic"]
)

In [34]:
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()

In [35]:
groq_model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x10f5bce90>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x10f5be000>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [36]:
prompt

PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, template='can you give me a detail explanation of {topic}')

In [37]:
parser

StrOutputParser()

In [38]:
chain = prompt | groq_model | parser

In [40]:
chain.invoke({"topic":"Machine Learning"})

'**Machine Learning: A Comprehensive Explanation**\n\nMachine learning is a subset of artificial intelligence (AI) that involves training algorithms to learn from data, enabling them to make predictions, classify objects, or make decisions without being explicitly programmed. In this explanation, we\'ll delve into the concepts, types, and applications of machine learning.\n\n**What is Machine Learning?**\n\nMachine learning is a type of AI that enables systems to learn from data, improve their performance, and make predictions or decisions based on that data. The goal of machine learning is to develop algorithms that can automatically learn and improve from experience without being explicitly programmed.\n\n**Key Concepts in Machine Learning**\n\n1. **Data**: Machine learning algorithms require large amounts of data to learn from. This data can be in the form of text, images, audio, or other types of data.\n2. **Algorithms**: Machine learning algorithms are the rules and procedures use

In [42]:
chain.get_graph().print_ascii()

     +-------------+       
     | PromptInput |       
     +-------------+       
            *              
            *              
            *              
    +----------------+     
    | PromptTemplate |     
    +----------------+     
            *              
            *              
            *              
      +----------+         
      | ChatGroq |         
      +----------+         
            *              
            *              
            *              
   +-----------------+     
   | StrOutputParser |     
   +-----------------+     
            *              
            *              
            *              
+-----------------------+  
| StrOutputParserOutput |  
+-----------------------+  


Sequential Chain

In [43]:
prompt1 = PromptTemplate(
    template = "get a detailed report on {topic}",
    input_variables = ["topic"]
)
prompt2 = PromptTemplate(
    template = "generate a 3 point summary on following {text}",
    input_variables = ["text"]
)

In [44]:
seq_chain = prompt1 | groq_model | parser | prompt2 | groq_model | parser
seq_chain.get_graph().print_ascii()

     +-------------+       
     | PromptInput |       
     +-------------+       
            *              
            *              
            *              
    +----------------+     
    | PromptTemplate |     
    +----------------+     
            *              
            *              
            *              
      +----------+         
      | ChatGroq |         
      +----------+         
            *              
            *              
            *              
   +-----------------+     
   | StrOutputParser |     
   +-----------------+     
            *              
            *              
            *              
+-----------------------+  
| StrOutputParserOutput |  
+-----------------------+  
            *              
            *              
            *              
    +----------------+     
    | PromptTemplate |     
    +----------------+     
            *              
            *              
            *       

In [45]:
seq_chain.invoke({"topic":"Machine Learning"})

"Here's a 3-point summary of the Machine Learning Report:\n\n1. **History and Types of Machine Learning**: Machine learning has its roots in the 1950s and gained momentum in the 1990s with the development of algorithms like decision trees, neural networks, and support vector machines. There are three primary types of machine learning: supervised learning, unsupervised learning, and reinforcement learning.\n\n2. **Applications and Techniques**: Machine learning has numerous applications across various industries, including computer vision, natural language processing, predictive maintenance, recommendation systems, and healthcare. Some popular machine learning techniques include decision trees, neural networks, support vector machines, random forests, and gradient boosting.\n\n3. **Challenges and Future of Machine Learning**: While machine learning has made tremendous progress, there are still challenges and limitations to consider, such as data quality, bias and fairness, explainabilit

Parallel Chains

In [52]:
prompt1=PromptTemplate(
    template="generate simple summary from the following text \n {text}",
    input_variables=["text"]
)

prompt2=PromptTemplate(
    template="generate 3 question and answer from the following text \n {text}",
    input_variables=["text"]
    )

prompt3=PromptTemplate(
    template="analysis the summary and qa and generate the 2 important quiz with 4 possible answer \n summary: {summary}, Q&A: {qa}",
    input_variables=["summary","qa"]
)

In [53]:
from langchain.schema.runnable import RunnableParallel
parallel_chain=RunnableParallel({
    "summary": prompt1 | groq_model | parser,
    "qa" : prompt2 | groq_model | parser
})

In [54]:
parallel_chain.get_graph().print_ascii()

          +---------------------------+            
          | Parallel<summary,qa>Input |            
          +---------------------------+            
                ***             ***                
              **                   **              
            **                       **            
+----------------+              +----------------+ 
| PromptTemplate |              | PromptTemplate | 
+----------------+              +----------------+ 
          *                             *          
          *                             *          
          *                             *          
    +----------+                  +----------+     
    | ChatGroq |                  | ChatGroq |     
    +----------+                  +----------+     
          *                             *          
          *                             *          
          *                             *          
+-----------------+            +-----------------+ 
| StrOutputP

In [55]:
merge_chain= prompt3 | groq_model | parser
chain = parallel_chain | merge_chain
chain.get_graph().print_ascii()

          +---------------------------+            
          | Parallel<summary,qa>Input |            
          +---------------------------+            
                ***             ***                
              **                   **              
            **                       **            
+----------------+              +----------------+ 
| PromptTemplate |              | PromptTemplate | 
+----------------+              +----------------+ 
          *                             *          
          *                             *          
          *                             *          
    +----------+                  +----------+     
    | ChatGroq |                  | ChatGroq |     
    +----------+                  +----------+     
          *                             *          
          *                             *          
          *                             *          
+-----------------+            +-----------------+ 
| StrOutputP

In [56]:
topic="""AI agent or simply agent), expands this concept by proactively pursuing goals, making decisions, and taking actions over extended periods, thereby exemplifying a novel form of digital agency.[1]

Intelligent agents can range from simple to highly complex. A basic thermostat or control system is considered an intelligent agent, as is a human being, or any other system that meets the same criteria—such as a firm, a state, or a biome.[2]

Intelligent agents operate based on an objective function, which encapsulates their goals. They are designed to create and execute plans that maximize the expected value of this function upon completion.[3] For example, a reinforcement learning agent has a reward function, which allows programmers to shape its desired behavior.[4] Similarly, an evolutionary algorithm's behavior is guided by a fitness function.[5]

Intelligent agents in artificial intelligence are closely related to agents in economics, and versions of the intelligent agent paradigm are studied in cognitive science, ethics, and the philosophy of practical reason, as well as in many interdisciplinary socio-cognitive modeling and computer social simulations.

Intelligent agents are often described schematically as abstract functional systems similar to computer programs. To distinguish theoretical models from real-world implementations, abstract descriptions of intelligent agents are called abstract intelligent agents. Intelligent agents are also closely related to software agents—autonomous computer programs that carry out tasks on behalf of users. They are also referred to using a term borrowed from economics: a "rational agent".[2]"""


In [57]:
result = chain.invoke({"text":topic})
result

'Based on the provided summary and Q&A, I have generated two important quiz questions with four possible answers each:\n\n**Quiz 1:** What is the main function of an objective function in intelligent agents?\n\nA) To minimize the performance of the agent\nB) To maximize the performance of the agent\nC) To shape the desired behavior of the agent\nD) To encapsulate the goals of the agent\n\nAnswer: D) To encapsulate the goals of the agent'

In [58]:
print(result)

Based on the provided summary and Q&A, I have generated two important quiz questions with four possible answers each:

**Quiz 1:** What is the main function of an objective function in intelligent agents?

A) To minimize the performance of the agent
B) To maximize the performance of the agent
C) To shape the desired behavior of the agent
D) To encapsulate the goals of the agent

Answer: D) To encapsulate the goals of the agent


In [59]:
template=PromptTemplate(
    template="generate a precise 3 point summary from given text /n {text}",
    input_variables=["text"]
)

In [61]:
chain = template | groq_model

In [63]:
chain_parser = template | groq_model | parser

In [65]:
chain.invoke({"text":topic})

AIMessage(content='Here is a 3-point summary of the given text:\n\n1. **Definition of Intelligent Agents**: An intelligent agent is a digital entity that expands the concept of agency by proactively pursuing goals, making decisions, and taking actions over extended periods, exemplifying a novel form of digital agency.\n\n2. **Types and Characteristics**: Intelligent agents can range from simple to complex, and operate based on an objective function that encapsulates their goals. They are designed to create and execute plans that maximize the expected value of this function.\n\n3. **Relationships and Applications**: Intelligent agents are related to agents in economics, and are studied in various fields including cognitive science, ethics, philosophy, and computer science. They are also closely related to software agents and are described schematically as abstract functional systems.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 158, 'prompt_tokens': 34

## Parsers

JSON Output Parser

In [66]:
text="""hi my name is Manas Agarwal my age is 25 and i am belong to bengaluru"""

In [67]:
from langchain_core.output_parsers import JsonOutputParser
parser=JsonOutputParser()
parser.get_format_instructions()

template=PromptTemplate(
    template="give me name, age and city from the provided text {text} \n {format_instructions}" ,
    input_variables=['text'],
    partial_variables={"format_instructions":parser.get_format_instructions()}
)

In [68]:
chain = template | groq_model | parser
chain.invoke({"text":text})

{'name': 'Manas Agarwal', 'age': 25, 'city': 'Bengaluru'}

Structure Output Parser

In [69]:
## Provides more control over format or structure of generated content 
## But it does not do data type validation ---> That's where pydantic output parser comes in 

from langchain.output_parsers import StructuredOutputParser, ResponseSchema

schema=[
    ResponseSchema(name="first_fact", description="first fact about text"),
    ResponseSchema(name="second_fact", description="second fact about text"),
    ResponseSchema(name="third_fact", description="third fact about text"),
]

parser=StructuredOutputParser.from_response_schemas(schema)

template3 = PromptTemplate(
    template='Give 3 fact about {topic} \n {format_instruction}',
    input_variables=['topic'],
    partial_variables={'format_instruction':parser.get_format_instructions()}
)


chain = template3 | groq_model | parser


result=chain.invoke({"topic":topic})

print(result)

{'first_fact': 'Intelligent agents can range from simple to highly complex, including basic thermostats, control systems, humans, firms, states, and biomes.', 'second_fact': 'Intelligent agents operate based on an objective function, which encapsulates their goals, and are designed to create and execute plans that maximize the expected value of this function upon completion.', 'third_fact': 'Intelligent agents in artificial intelligence are closely related to agents in economics and are studied in various fields, including cognitive science, ethics, philosophy, and socio-cognitive modeling, as well as computer social simulations.'}


In [70]:
result

{'first_fact': 'Intelligent agents can range from simple to highly complex, including basic thermostats, control systems, humans, firms, states, and biomes.',
 'second_fact': 'Intelligent agents operate based on an objective function, which encapsulates their goals, and are designed to create and execute plans that maximize the expected value of this function upon completion.',
 'third_fact': 'Intelligent agents in artificial intelligence are closely related to agents in economics and are studied in various fields, including cognitive science, ethics, philosophy, and socio-cognitive modeling, as well as computer social simulations.'}

Pydantic OutputParser

In [ ]:
from pydantic import BaseModel, Field
from langchain_core.output_parsers import PydanticOutputParser
import json
import random

text="""hi my name is Manas Agarwal my age is 25 and i am belong to bengaluru"""

class Person(BaseModel):
    name:str=Field(description="name of person")
    age:int=Field(gt=18,description="age of person")
    city:str=Field(description="name of the city where the person is located")
    
parser=PydanticOutputParser(pydantic_object=Person)

template=PromptTemplate(
    template="give me name, age and city from the provided text {text} \n {format_instructions}" ,
    input_variables=['text'],
    partial_variables={"format_instructions":parser.get_format_instructions()}
)

chain = template | groq_model | parser

result=chain.invoke({"text":text})


result

## Basic RAG application

In [93]:
pip install --upgrade --quiet  libdeeplake

Note: you may need to restart the kernel to use updated packages.


In [ ]:
!pip install langchain-community 

In [7]:
import deeplake
deeplake.__version__ = '3.6.2'

In [52]:
from langchain.document_loaders import TextLoader

text =""" Google opens up its AI language model PaLM to challenge OpenAI and GPT-3 Google 
offers developers access to one of its most advanced AI language models: PaLM. The search 
giant is launching an API for PaLM alongside a number of AI enterprise tools it says will 
help businesses generate text, images, code, videos, audio, and more from simple natural 
language prompts. 

PaLM is a large language model, or LLM, similar to the GPT series created 
by OpenAI or Meta's LLaMA family of models. Google first announced PaLM in April 2022. Like 
other LLMs, PaLM is a flexible system that can potentially carry out all sorts of text 
generation and editing tasks. You could train PaLM to be a conversational chatbot like 
ChatGPT, for example, or you could use it for tasks like summarizing text or even writing code. 
(It's similar to features Google also announced today for its Workspace apps like Google Docs 
and Gmail.)
"""

with open("file.txt",'w') as file:
    file.write(text)
    
loader = TextLoader("file.txt")
docs_from_file = loader.load()
docs_from_file

[Document(metadata={'source': 'file.txt'}, page_content=" Google opens up its AI language model PaLM to challenge OpenAI and GPT-3 Google \noffers developers access to one of its most advanced AI language models: PaLM. The search \ngiant is launching an API for PaLM alongside a number of AI enterprise tools it says will \nhelp businesses generate text, images, code, videos, audio, and more from simple natural \nlanguage prompts. \n\nPaLM is a large language model, or LLM, similar to the GPT series created \nby OpenAI or Meta's LLaMA family of models. Google first announced PaLM in April 2022. Like \nother LLMs, PaLM is a flexible system that can potentially carry out all sorts of text \ngeneration and editing tasks. You could train PaLM to be a conversational chatbot like \nChatGPT, for example, or you could use it for tasks like summarizing text or even writing code. \n(It's similar to features Google also announced today for its Workspace apps like Google Docs \nand Gmail.)\n")]

In [53]:
len(docs_from_file)

1

In [54]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=50, chunk_overlap=20)
docs = text_splitter.split_documents(docs_from_file)

Created a chunk of size 372, which is longer than the specified 50


In [55]:
len(docs)

2

In [15]:
from langchain_huggingface import HuggingFaceEmbeddings
embedding_model = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

In [ ]:
!pip install pinecone

In [12]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key="pcsk_6B7Uez_S2jqvMMHdU2c7SpVjHtMFrXibiRejE1VfKewM8mTAFUXyF2hZ4U5N3gKEdAvxdT")

In [30]:
index_name = "quickstart02"

pc.create_index(
    name=index_name,
    dimension=384, # Replace with your model dimensions
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

{
    "name": "quickstart02",
    "metric": "cosine",
    "host": "quickstart02-scup0jf.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [31]:
index = pc.Index(index_name)

In [ ]:
!pip install langchain_pinecone

In [ ]:
from uuid import uuid4
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embedding_model)

uuids = [str(uuid4()) for _ in range(len(docs))]
vector_store.add_documents(documents=docs, ids=uuids)

In [34]:
print(index.describe_index_stats())

{'dimension': 384,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'': {'vector_count': 1}},
 'total_vector_count': 1,
 'vector_type': 'dense'}


In [56]:
retriever = vector_store.as_retriever()
retriever.get_relevant_documents("How Google plans to challenge OpenAI?")

[Document(id='b5deb344-a1de-401d-ab97-df775d723b33', metadata={'source': 'file.txt'}, page_content='Google opens up its AI language model PaLM to challenge OpenAI and GPT-3 Google offers developers access to one of its most advanced AI language models: PaLM. The search giant is launching an API for PaLM alongside a number of AI enterprise tools it says will help businesses "generate text, images, code, videos, audio, and more from simple natural language prompts."\nPaLM is a large language model, or LLM, similar to the GPT series created by OpenAI or Meta\'s LLaMA family of models. Google first announced PaLM in April 2022. Like other LLMs, PaLM is a flexible system that can potentially carry out all sorts of text generation and editing tasks. You could train PaLM to be a conversational chatbot like ChatGPT, for example, or you could use it for tasks like summarizing text or even writing code. (It\'s similar to features Google also announced today for its Workspace apps like Google Doc

In [36]:
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA

groq_model=ChatGroq(model="llama-3.1-8b-instant")
qa_chain = RetrievalQA.from_chain_type(llm=groq_model, 
                                       chain_type= "stuff", 
                                       retriever=retriever)
query = "How Google plans to challenge OpenAI?" 
response = qa_chain.run(query)
print(response)

/var/folders/9d/v_vmnb4s20xddx4_8tgtprrh0000gn/T/ipykernel_45947/1506314546.py:9: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa_chain.run(query)


I don't have the most up-to-date information on Google's plans to challenge OpenAI. However, based on my knowledge, Google has been actively developing its own language model technology, called LaMDA (Language Model for Dialogue Applications). LaMDA is a conversational AI model that aims to engage in natural-sounding conversations with humans.

In 2021, Google announced that LaMDA would be a key part of its plan to develop more advanced AI models. LaMDA has been shown to be capable of generating human-like responses to a wide range of questions and topics.

Additionally, Google has been investing heavily in its AI research and development, with a focus on areas such as natural language processing, computer vision, and reinforcement learning. The company has also been making significant hires in the AI space, including the acquisition of companies like DeepMind, which developed the AlphaGo AI that defeated a human world champion in Go.

While I don't have specific information on Google'

In [57]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

compressor = LLMChainExtractor.from_llm(groq_model) 
compression_retriever = ContextualCompressionRetriever(
    base_compressor = compressor,
    base_retriever = retriever
)

In [58]:
retrieved_docs = compression_retriever.get_relevant_documents("How Google plans to challenge OpenAI?") 
print(retrieved_docs[0].page_content)

Google opens up its AI language model PaLM to challenge OpenAI and GPT-3 Google offers developers access to one of its most advanced AI language models: PaLM. The search giant is launching an API for PaLM alongside a number of AI enterprise tools it says will help businesses "generate text, images, code, videos, audio, and more from simple natural language prompts."
PaLM is a large language model, or LLM, similar to the GPT series created by OpenAI or Meta's LLaMA family of models.


In [59]:
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA

groq_model=ChatGroq(model="llama-3.1-8b-instant")
qa_chain = RetrievalQA.from_chain_type(llm=groq_model, 
                                       chain_type= "stuff", 
                                       retriever=compression_retriever)
query = "How Google plans to challenge OpenAI?" 
response = qa_chain.run(query)
print(response)

Google plans to challenge OpenAI by offering developers access to one of its most advanced AI language models, PaLM, through an API. This will allow businesses to use PaLM to generate various content such as text, images, code, videos, and audio from simple natural language prompts. 

Google's move suggests that the company is positioning its AI enterprise tools, including the PaLM API, as a direct competitor to OpenAI's GPT-3 model and other similar offerings.


## Structured Output

In [1]:
from typing import TypedDict

class Person(TypedDict):
    name : str
    age : int
    
new_person = Person(
    name = "Manas Agarwal",
    age = 24
)
print(new_person)

{'name': 'Manas Agarwal', 'age': 24}


In [2]:
new_person = Person(
    name = "Manas Agarwal",
    age = '24'
)

print(new_person)

{'name': 'Manas Agarwal', 'age': '24'}


In [16]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv
from typing import TypedDict, Annotated, List, Optional, Literal

load_dotenv()

groq_model = ChatGroq(model = "llama-3.1-8b-instant",
                      temperature = 0,
                      max_retries = 2)

## Schema

class Review(TypedDict):
    summary : Annotated[str,"Summary review in 1 point"]
    sentiment : Annotated[Literal["pos","neg"],"sentiment of review which is one of following == Positive -> pos or Negative -> neg"]
    topic : Annotated[List[str],"Key Themes or list of topics talked about in review"]
    pros : Annotated[Optional[List[str]],"List of pros of using the product"]
    cons : Annotated[Optional[List[str]],"List of cons of using the product"]
    name : Annotated[Optional[str],"Name of the person who reviewed the product"]
    
structured_model = groq_model.with_structured_output(Review)

result = structured_model.invoke("I had high hopes for Indulekha Bringha Shampoo after hearing all the hype about its Ayurvedic formula and promises to reduce hair fall. Unfortunately, it left me completely disappointed. After using it for a few weeks, I saw no improvement in my hair fall— if anything, it felt worse. The shampoo has a thick, herbal texture and a strong smell that’s not exactly pleasant, but I was willing to overlook that if it worked. It didn’t. My hair felt dry and rough after every wash, and the scalp irritation I started experiencing was the final straw. For a product that claims to be a “proprietary Ayurvedic medicine,” it didn’t deliver any noticeable benefits. It’s overpriced for what it is—just a basic cleanser that doesn’t live up to its claims. I won’t be repurchasing. Reviewed by Manas Agarwal")
result

{'cons': ["Ayurvedic formula didn't work",
  'thick herbal texture',
  'strong smell',
  'dry and rough hair',
  'scalp irritation',
  'overpriced'],
 'name': 'Manas Agarwal',
 'pros': [],
 'sentiment': 'neg',
 'summary': "Disappointed with the shampoo's performance and claims",
 'topic': ['hair fall',
  'Ayurvedic formula',
  'hair care',
  'shampoo',
  'product review']}

In [18]:
from pydantic import BaseModel, EmailStr, Field
from typing import Annotated, List, Optional, Dict

class Student(BaseModel):
    name: str = Field(max_length=50,description="Name of Student")
    CGPA : float = Field(gt=0,lt=10,description="Decimanl value indicating total Cumulative GPA over his graduation")
    age: Optional[int] = Field(gt=0, default=None, description="Age of student")

In [22]:
from typing import Annotated, List, Optional, Literal
from pydantic import BaseModel,Field

class Review(BaseModel):
    summary : Annotated[str,Field(description="Summary review in 1 point")]
    sentiment : Annotated[Literal["pos","neg"],Field(description="sentiment of review which is one of following == Positive -> pos or Negative -> neg")]
    key_themes : Annotated[List[str],Field(description="Key Themes or list of topics talked about in review")]
    pros : Annotated[Optional[List[str]],Field(default=None,description="List of pros of using the product")]
    cons : Annotated[Optional[List[str]],Field(default=None,description = "List of cons of using the product")]
    name : Annotated[Optional[str],Field(default= None, description ="Name of the person who reviewed the product")]
    
structured_model = groq_model.with_structured_output(Review)

result = structured_model.invoke("I had high hopes for Indulekha Bringha Shampoo after hearing all the hype about its Ayurvedic formula and promises to reduce hair fall. Unfortunately, it left me completely disappointed. After using it for a few weeks, I saw no improvement in my hair fall— if anything, it felt worse. The shampoo has a thick, herbal texture and a strong smell that’s not exactly pleasant, but I was willing to overlook that if it worked. It didn’t. My hair felt dry and rough after every wash, and the scalp irritation I started experiencing was the final straw. For a product that claims to be a “proprietary Ayurvedic medicine,” it didn’t deliver any noticeable benefits. It’s overpriced for what it is—just a basic cleanser that doesn’t live up to its claims. I won’t be repurchasing.")
result

Review(summary='I had high hopes for Indulekha Bringha Shampoo after hearing all the hype about its Ayurvedic formula and promises to reduce hair fall. Unfortunately, it left me completely disappointed.', sentiment='neg', key_themes=['Ayurvedic formula', 'hair fall', 'hair texture', 'scalp irritation', 'price'], pros=None, cons=['thick herbal texture', 'strong smell', 'dry and rough hair', 'scalp irritation', 'overpriced'], name='null')

In [23]:
result.model_dump()

{'summary': 'I had high hopes for Indulekha Bringha Shampoo after hearing all the hype about its Ayurvedic formula and promises to reduce hair fall. Unfortunately, it left me completely disappointed.',
 'sentiment': 'neg',
 'key_themes': ['Ayurvedic formula',
  'hair fall',
  'hair texture',
  'scalp irritation',
  'price'],
 'pros': None,
 'cons': ['thick herbal texture',
  'strong smell',
  'dry and rough hair',
  'scalp irritation',
  'overpriced'],
 'name': 'null'}

In [ ]:
{
    "title" : 'Student',
    "description" : "Description about Student",
    "type" : "object",
    "properties" : {
        "name" : "string",
        "age" : "int",
    },
    "required" : ["name"]
}

In [28]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate
parser = JsonOutputParser()

template = ChatPromptTemplate(
    
    messages= [("system","Always {format_instruction}"),
               ("human","Give me a fictional person's name, age, city and profession")],
    
    input_variables = [],
    partial_variables = {"format_instruction":parser.get_format_instructions()}
)

prompt = template.invoke({})
prompt

ChatPromptValue(messages=[SystemMessage(content='Always Return a JSON object.', additional_kwargs={}, response_metadata={}), HumanMessage(content="Give me a fictional person's name, age, city and profession", additional_kwargs={}, response_metadata={})])

In [40]:
prompt.messages

[SystemMessage(content='Always Return a JSON object.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content="Give me a fictional person's name, age, city and profession", additional_kwargs={}, response_metadata={})]

In [41]:
prompt.to_messages()

[SystemMessage(content='Always Return a JSON object.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content="Give me a fictional person's name, age, city and profession", additional_kwargs={}, response_metadata={})]

In [43]:
print(prompt.to_string())

System: Always Return a JSON object.
Human: Give me a fictional person's name, age, city and profession


In [39]:
from langchain_groq import ChatGroq
groq_model = ChatGroq(model="llama-3.1-8b-instant",temperature=1.5)
chain = template | groq_model | parser

chain.invoke({})

{'name': 'Evelyn Stonebridge',
 'age': 32,
 'city': 'Los Angeles',
 'profession': 'Software Engineer'}

In [95]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain_core.prompts import PromptTemplate

schema = [
    ResponseSchema(name = "fact_1", description="Fact 1 about the topic"),
    ResponseSchema(name = "fact_2", description="Fact 2 about the topic"),
    ResponseSchema(name = "fact_3", description="Fact 3 about the topic")
]

parser = StructuredOutputParser.from_response_schemas(schema)

groq_model = ChatGroq(model="deepseek-r1-distill-llama-70b")

prompt_template = PromptTemplate(
    template= "Give me 3 facts about {topic}\n{format_instructions}.",
    input_variables = ['topic'],
    partial_variables = {"format_instructions":parser.get_format_instructions()}
)

prompt_template.invoke({"topic":"ultrasound during pregnancy"})

StringPromptValue(text='Give me 3 facts about ultrasound during pregnancy\nThe output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"fact_1": string  // Fact 1 about the topic\n\t"fact_2": string  // Fact 2 about the topic\n\t"fact_3": string  // Fact 3 about the topic\n}\n```.')

In [97]:
prompt = prompt_template.invoke({"topic":"ultrasound during pregnancy"})
result = groq_model.invoke(prompt)

In [98]:
print(result.content)

<think>
Okay, so the user is asking for three facts about ultrasound during pregnancy. They also provided a specific format using markdown code. I need to make sure I follow their instructions precisely.

First, I should recall what an ultrasound is. It's a prenatal test that uses high-frequency sound waves to create images of the baby inside the uterus. That's a good starting point for fact one.

Next, I remember that ultrasounds are used for multiple purposes. They can confirm the pregnancy, check the heartbeat, determine the due date, and even identify the baby's position. That can be the second fact.

For the third fact, I should mention that there are different types of ultrasounds. The standard 2D ultrasound is common, but there are also 3D and 4D options which provide more detailed images. This adds value as it shows the variety available.

I need to structure these facts into a JSON schema within a markdown code snippet. Making sure each fact is concise and informative. Also, I

In [99]:
output = parser.parse(result.content)

In [100]:
output

{'fact_1': 'Ultrasound during pregnancy uses high-frequency sound waves to create images of the developing fetus, allowing healthcare providers to monitor growth and development.',
 'fact_2': 'The first trimester ultrasound can confirm pregnancy, detect the heartbeat, and estimate the due date, while later ultrasounds can check for fetal position and any potential abnormalities.',
 'fact_3': 'There are different types of ultrasounds, including standard 2D, 3D, and 4D ultrasounds, with 3D and 4D providing more detailed and dynamic images of the fetus.'}

In [77]:
from pydantic import BaseModel, Field
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate

class Person(BaseModel):
    name : str = Field(description="Name of the person")
    age : int = Field(description="Age of the person",gt=18)
    city : str = Field(description="This is the city where person lives")
    
parser = PydanticOutputParser(pydantic_object=Person)

template = PromptTemplate(
    template = "Generate name, age and city of fictional person who belongs to {country}\n{format_instructions}",
    input_variables= ["country"],
    partial_variables={"format_instructions":parser.get_format_instructions()}
)

prompt = template.invoke({"country":"India"})
prompt

StringPromptValue(text='Generate name, age and city of fictional person who belongs to India\nThe output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"name": {"description": "Name of the person", "title": "Name", "type": "string"}, "age": {"description": "Age of the person", "exclusiveMinimum": 18, "title": "Age", "type": "integer"}, "city": {"description": "This is the city where person lives", "title": "City", "type": "string"}}, "required": ["name", "age", "city"]}\n```')

In [92]:
groq_model = ChatGroq(model="deepseek-r1-distill-llama-70b")

chain = template | groq_model | parser
chain.invoke({"country":"India"})

Person(name='Rajesh Kumar', age=32, city='Mumbai')

In [93]:
chain.invoke({"country":"USA"})

Person(name='Emily Johnson', age=32, city='New York City')

In [94]:
chain.invoke({"country":"England"})

Person(name='Emily Carter', age=27, city='London')